## Train with imgaug generator

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm
from skimage.transform import resize
import keras
import cv2
%matplotlib inline

import train_util
import util
from image_generator import Generator
%load_ext autoreload
%autoreload 2


/root/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Make {'image_id':depth} dictionary and train-val split

In [2]:
master_df = pd.read_csv('./data/depth_prop_cls_bad_mask.csv')
id_depth_df = master_df[['id', 'z']].set_index('id')['z']
depth_dict = id_depth_df.to_dict()

In [3]:
# data_type is 'train', 'test' and 'bad_train'
train_val_df = master_df[(master_df['data_type'] == 'train')]
ids = train_val_df['id'].values
classes = train_val_df['salt_propotion_class'].values

In [4]:
from sklearn.cross_validation import train_test_split
train_ids, val_ids = train_test_split(ids, test_size=0.2, random_state=43, stratify=classes)

In [5]:
# make generator
batch_size = 32
tgs_generator = Generator(train_ids = train_ids, depth_dict=depth_dict, val_ids=val_ids, batch_size=batch_size)

In [6]:
# steps_per_epoch
steps_per_epoch = len(train_ids) //  batch_size
validation_steps = len(val_ids) //  batch_size
# validation batch will be all validation
# validation_steps = 1
print(steps_per_epoch)
print(validation_steps)

97
24


### Make val data

In [7]:
# X_val = np.zeros((len(val_ids), 128, 128, 3), dtype=np.float32)
# X_feat_val = np.zeros((len(val_ids), 1), dtype=np.float32)
# Y_val_true = np.zeros((len(val_ids), 128, 128, 1), dtype=np.float32)

# for i, im_id in enumerate(val_ids):
# #     image
#     im = cv2.imread('./data/train/images/' + im_id + '.png')
#     im = cv2.resize(im, (128, 128))
#     im = np.array(im, dtype='float32')
#     im /= 255
#     X_val[i] = im
#     #     feature
#     X_feat_val[i] = depth_dict[im_id]
# #     mask
#     mask = cv2.imread('./data/train/masks/' + im_id + '.png')
#     mask = cv2.resize(mask, (128, 128), interpolation=cv2.INTER_NEAREST)
#     mask = np.array((mask > 127.5), dtype='float32')
#     mask = np.expand_dims(mask[:, :, 0], axis=-1)
#     Y_val_true[i] = mask

# val_gen = ([X_val, X_feat_val], Y_val_true)

### Model

In [8]:
unet_m = util.unet_model(min_filter_num=16, with_vec=True)

## Train

In [ ]:
optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=0.0, nesterov=True) 
# Todo Adam 
# train_util.train(unet_m, tgs_generator.generate(True), tgs_generator.generate(False), loss=train_util.dice_p_bce, metrics=[train_util.dice_coef, 'binary_accuracy', train_util.true_positive_rate, train_util.iou_coef], epochs=3000, steps_per_epoch=steps_per_epoch, validation_steps=validation_steps)
train_util.train(unet_m, 
                 tgs_generator.generate(True), 
                 tgs_generator.generate(False), 
                 metrics=[train_util.dice_coef, 'binary_accuracy', train_util.true_positive_rate, train_util.iou_coef, train_util.mean_iou_threshold],
                 epochs=3000, steps_per_epoch=steps_per_epoch, 
                 validation_steps=validation_steps, 
                 optimizer=optimizer)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
encoder_block_1 (Conv2D)        (None, 128, 128, 16) 448         input_layer[0][0]                
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 128, 128, 16) 64          encoder_block_1[0][0]            
__________________________________________________________________________________________________
elu_1 (ELU)                     (None, 128, 128, 16) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
dropout_1 

Epoch 1/3000
 - 64s - loss: 0.6160 - dice_coef: 0.1667 - binary_accuracy: 0.6981 - true_positive_rate: 0.1221 - iou_coef: 0.0397 - mean_iou_threshold: 0.0032 - val_loss: 0.5613 - val_dice_coef: 0.1865 - val_binary_accuracy: 0.7625 - val_true_positive_rate: 3.5352e-04 - val_iou_coef: 0.2268 - val_mean_iou_threshold: 0.2266
Finished epoch: 0
{'val_mean_iou_threshold': 0.2265625, 'val_binary_accuracy': 0.7625198364257812, 'true_positive_rate': 0.12208430918195218, 'mean_iou_threshold': 0.0032216495349427165, 'val_iou_coef': 0.2268033673365911, 'val_dice_coef': 0.18654175971945128, 'dice_coef': 0.16668962702615975, 'val_true_positive_rate': 0.0003535198396396784, 'loss': 0.6159536777083406, 'val_loss': 0.5612996543447176, 'binary_accuracy': 0.6981483341492328, 'iou_coef': 0.03972704393099681}

Epoch 00001: val_loss improved from inf to 0.56130, saving model to ./log/2018_0808_2045/best_weights.hdf5
Epoch 2/3000
 - 60s - loss: 0.5265 - dice_coef: 0.1524 - binary_accuracy: 0.7632 - true_posi

 - 64s - loss: 0.3152 - dice_coef: 0.2831 - binary_accuracy: 0.8723 - true_positive_rate: 0.6972 - iou_coef: 0.4826 - mean_iou_threshold: 0.4016 - val_loss: 0.5513 - val_dice_coef: 0.3397 - val_binary_accuracy: 0.7598 - val_true_positive_rate: 0.9234 - val_iou_coef: 0.4961 - val_mean_iou_threshold: 0.4013
Finished epoch: 10
{'val_mean_iou_threshold': 0.4013020849476258, 'val_binary_accuracy': 0.7598224480946859, 'true_positive_rate': 0.6971579123403608, 'mean_iou_threshold': 0.4016108237130126, 'val_iou_coef': 0.4960663045446078, 'val_dice_coef': 0.33970339968800545, 'dice_coef': 0.2831202587976898, 'val_true_positive_rate': 0.9234162444869677, 'loss': 0.3152181837976593, 'val_loss': 0.5512934376796087, 'lr': 0.01, 'binary_accuracy': 0.8722977982353919, 'iou_coef': 0.48258335765489596}

Epoch 00011: val_loss did not improve
Epoch 12/3000
 - 60s - loss: 0.2960 - dice_coef: 0.2896 - binary_accuracy: 0.8836 - true_positive_rate: 0.7150 - iou_coef: 0.5110 - mean_iou_threshold: 0.4371 - val

Epoch 21/3000
 - 61s - loss: 0.2713 - dice_coef: 0.3034 - binary_accuracy: 0.8941 - true_positive_rate: 0.7361 - iou_coef: 0.5459 - mean_iou_threshold: 0.4765 - val_loss: 0.4563 - val_dice_coef: 0.3544 - val_binary_accuracy: 0.8199 - val_true_positive_rate: 0.9052 - val_iou_coef: 0.5559 - val_mean_iou_threshold: 0.4734
Finished epoch: 20
{'val_mean_iou_threshold': 0.47343750422199565, 'val_binary_accuracy': 0.8198846181233724, 'true_positive_rate': 0.7360535872351263, 'mean_iou_threshold': 0.4765463938418123, 'val_iou_coef': 0.5558713612457117, 'val_dice_coef': 0.3544146741429965, 'dice_coef': 0.3033839041732021, 'val_true_positive_rate': 0.9052025775114695, 'loss': 0.27128969424778654, 'val_loss': 0.4563389867544174, 'lr': 0.01, 'binary_accuracy': 0.89410284376636, 'iou_coef': 0.5458799443908573}

Epoch 00021: val_loss did not improve
Epoch 22/3000
 - 60s - loss: 0.2711 - dice_coef: 0.3051 - binary_accuracy: 0.8947 - true_positive_rate: 0.7417 - iou_coef: 0.5495 - mean_iou_threshold: 

 - 59s - loss: 0.2542 - dice_coef: 0.3171 - binary_accuracy: 0.9020 - true_positive_rate: 0.7464 - iou_coef: 0.5664 - mean_iou_threshold: 0.5011 - val_loss: 0.4883 - val_dice_coef: 0.3667 - val_binary_accuracy: 0.8303 - val_true_positive_rate: 0.9148 - val_iou_coef: 0.5456 - val_mean_iou_threshold: 0.4689
Finished epoch: 30
{'val_mean_iou_threshold': 0.4688802088300387, 'val_binary_accuracy': 0.830260435740153, 'true_positive_rate': 0.7464124388301495, 'mean_iou_threshold': 0.501095359165644, 'val_iou_coef': 0.5456222891807556, 'val_dice_coef': 0.36669451867540676, 'dice_coef': 0.31706949653699223, 'val_true_positive_rate': 0.9148137594262759, 'loss': 0.2542368863354024, 'val_loss': 0.48833097393314046, 'lr': 0.01, 'binary_accuracy': 0.9020151551236811, 'iou_coef': 0.5664128525969908}

Epoch 00031: val_loss did not improve
Epoch 32/3000
 - 61s - loss: 0.2518 - dice_coef: 0.3153 - binary_accuracy: 0.9008 - true_positive_rate: 0.7416 - iou_coef: 0.5644 - mean_iou_threshold: 0.4976 - val_

...

 - 65s - loss: 0.1910 - dice_coef: 0.3573 - binary_accuracy: 0.9282 - true_positive_rate: 0.8183 - iou_coef: 0.6287 - mean_iou_threshold: 0.5731 - val_loss: 0.2529 - val_dice_coef: 0.4108 - val_binary_accuracy: 0.8961 - val_true_positive_rate: 0.9396 - val_iou_coef: 0.5972 - val_mean_iou_threshold: 0.5368
Finished epoch: 100
{'val_mean_iou_threshold': 0.5368489573399226, 'val_binary_accuracy': 0.8961335023244222, 'true_positive_rate': 0.8182602807418587, 'mean_iou_threshold': 0.5731314417627669, 'val_iou_coef': 0.5971936906377474, 'val_dice_coef': 0.41075095906853676, 'dice_coef': 0.3572786911246703, 'val_true_positive_rate': 0.9396121303240458, 'loss': 0.19104998551078678, 'val_loss': 0.2528766921410958, 'lr': 7.999999e-05, 'binary_accuracy': 0.9282248978762283, 'iou_coef': 0.6287230363826162}

Epoch 00101: val_loss did not improve
Epoch 102/3000
 - 62s - loss: 0.2010 - dice_coef: 0.3570 - binary_accuracy: 0.9219 - true_positive_rate: 0.8096 - iou_coef: 0.6127 - mean_iou_threshold: 0.

 - 65s - loss: 0.1949 - dice_coef: 0.3573 - binary_accuracy: 0.9256 - true_positive_rate: 0.8124 - iou_coef: 0.6319 - mean_iou_threshold: 0.5734 - val_loss: 0.2512 - val_dice_coef: 0.4103 - val_binary_accuracy: 0.8967 - val_true_positive_rate: 0.9401 - val_iou_coef: 0.5972 - val_mean_iou_threshold: 0.5366
Finished epoch: 110
{'val_mean_iou_threshold': 0.5365885396798452, 'val_binary_accuracy': 0.8966566721598307, 'true_positive_rate': 0.8123822365839457, 'mean_iou_threshold': 0.5733569587014385, 'val_iou_coef': 0.597169807801644, 'val_dice_coef': 0.4102884617944558, 'dice_coef': 0.35727954248792115, 'val_true_positive_rate': 0.94012783964475, 'loss': 0.19490115498144603, 'val_loss': 0.25118889721731347, 'lr': 1.5999998e-05, 'binary_accuracy': 0.9256410107170183, 'iou_coef': 0.6319280134648392}

Epoch 00111: val_loss did not improve
Epoch 112/3000
 - 63s - loss: 0.1888 - dice_coef: 0.3582 - binary_accuracy: 0.9284 - true_positive_rate: 0.8198 - iou_coef: 0.6222 - mean_iou_threshold: 0.5

 - 64s - loss: 0.1987 - dice_coef: 0.3574 - binary_accuracy: 0.9231 - true_positive_rate: 0.8133 - iou_coef: 0.6179 - mean_iou_threshold: 0.5595 - val_loss: 0.2531 - val_dice_coef: 0.4106 - val_binary_accuracy: 0.8955 - val_true_positive_rate: 0.9404 - val_iou_coef: 0.5971 - val_mean_iou_threshold: 0.5367
Finished epoch: 120
{'val_mean_iou_threshold': 0.5367187534769376, 'val_binary_accuracy': 0.895537535349528, 'true_positive_rate': 0.8133299750151094, 'mean_iou_threshold': 0.5595038640744907, 'val_iou_coef': 0.5971006962160269, 'val_dice_coef': 0.4105992230276267, 'dice_coef': 0.35737684353725196, 'val_true_positive_rate': 0.9404346098502477, 'loss': 0.1987419471759157, 'val_loss': 0.25310648294786614, 'lr': 3.1999996e-06, 'binary_accuracy': 0.923141774443007, 'iou_coef': 0.6179258098307344}

Epoch 00121: val_loss did not improve
Epoch 122/3000
 - 61s - loss: 0.1887 - dice_coef: 0.3581 - binary_accuracy: 0.9288 - true_positive_rate: 0.8215 - iou_coef: 0.6256 - mean_iou_threshold: 0.5

 - 63s - loss: 0.1885 - dice_coef: 0.3576 - binary_accuracy: 0.9288 - true_positive_rate: 0.8196 - iou_coef: 0.6261 - mean_iou_threshold: 0.5695 - val_loss: 0.2517 - val_dice_coef: 0.4105 - val_binary_accuracy: 0.8961 - val_true_positive_rate: 0.9399 - val_iou_coef: 0.5997 - val_mean_iou_threshold: 0.5392
Finished epoch: 130
{'val_mean_iou_threshold': 0.5391927113135656, 'val_binary_accuracy': 0.896127462387085, 'true_positive_rate': 0.819578815366804, 'mean_iou_threshold': 0.5694909787055144, 'val_iou_coef': 0.5996862960358461, 'val_dice_coef': 0.4104856240252654, 'dice_coef': 0.3576182862839748, 'val_true_positive_rate': 0.9399318620562553, 'loss': 0.18850947140725619, 'val_loss': 0.2517195548862219, 'lr': 6.3999994e-07, 'binary_accuracy': 0.9288310218103153, 'iou_coef': 0.6260898131685159}

Epoch 00131: val_loss did not improve
Epoch 132/3000
 - 60s - loss: 0.1985 - dice_coef: 0.3544 - binary_accuracy: 0.9231 - true_positive_rate: 0.8101 - iou_coef: 0.6062 - mean_iou_threshold: 0.54

 - 59s - loss: 0.1931 - dice_coef: 0.3573 - binary_accuracy: 0.9255 - true_positive_rate: 0.8153 - iou_coef: 0.6204 - mean_iou_threshold: 0.5612 - val_loss: 0.2511 - val_dice_coef: 0.4101 - val_binary_accuracy: 0.8966 - val_true_positive_rate: 0.9398 - val_iou_coef: 0.5984 - val_mean_iou_threshold: 0.5382
Finished epoch: 140
{'val_mean_iou_threshold': 0.5381510431567827, 'val_binary_accuracy': 0.8965617020924886, 'true_positive_rate': 0.8152863272686595, 'mean_iou_threshold': 0.5611791269680888, 'val_iou_coef': 0.5983584312101206, 'val_dice_coef': 0.41009438037872314, 'dice_coef': 0.35732833749240206, 'val_true_positive_rate': 0.9397893274823824, 'loss': 0.19312909537369444, 'val_loss': 0.25106803824504215, 'lr': 1.2799998e-07, 'binary_accuracy': 0.9255101312067091, 'iou_coef': 0.6204290343928582}

Epoch 00141: val_loss did not improve
Epoch 142/3000
 - 60s - loss: 0.1977 - dice_coef: 0.3566 - binary_accuracy: 0.9244 - true_positive_rate: 0.8077 - iou_coef: 0.6278 - mean_iou_threshold:

## Check iou_coef

In [7]:
from keras.models import load_model
model = load_model('./log/2018_0807_2002/best_weights.hdf5', custom_objects={
    'mean_iou': train_util.mean_iou, 
    'dice_p_bce':train_util.dice_p_bce, 
    'dice_coef':train_util.dice_coef,
    'true_positive_rate':train_util.true_positive_rate,
    'iou_coef':train_util.iou_coef
})

In [29]:
preds = model.predict([X_val, X_feat_val])

In [30]:
preds_t = (preds > 0.5).astype(np.uint8)

In [33]:
ious = []
for i in range(0, len(preds_t)):
    y_true = Y_val_true[i]
    y_pred = preds_t[i]
    ious.append(train_util.compute_iou(y_true, y_pred))

In [36]:
np.mean(ious)

0.7901850010723667

In [37]:
len(train_util.BAD_MASKS)

113

In [41]:
113/(len(preds) * 5)

0.02904884318766067

In [45]:
0.79/1.02

0.7745098039215687

In [52]:
iou = train_util.iou_coef(Y_val_true, np.float32(preds_t))

In [58]:
from keras import backend as K
K.get_value(K.mean((iou)))

0.78863376